# 🚀 02: LangGraph 실무 패턴 (EXAONE 실전)

## 💡 01번까지 배운 메커니즘을 실제 LLM으로 구현하자!

### 📚 지금까지의 학습 경로
```
00번: State, Node, Graph 기본 개념
  ↓
01번: LangGraph 메커니즘 (Command, Annotated, interrupt 도구)
  ↓
02번: 실무 적용 (EXAONE으로 실제 비즈니스 문제 해결) ← 지금 여기!
```

### 🔍 01번 vs 02번 차이

#### 01번 (advanced-integration.ipynb)
- 🔧 **메커니즘 중심**: Command, Annotated, interrupt가 **어떻게 작동하는지**
- 📐 **구조 이해**: LangGraph 내부 동작 원리
- 💡 **간단한 예제**: 문자열 기반, Mock 데이터
- 🎯 **목표**: "LangGraph 도구를 어떻게 쓰는가"

#### 02번 (지금 노트북)
- 🚀 **실무 패턴 중심**: Routing, Summarization 등 **비즈니스 문제 해결**
- 🤖 **실제 LLM 사용**: EXAONE이 진짜로 추론하고 응답 생성
- 📊 **다양한 시나리오**: 이메일 응답, 고객 문의, 대화 요약
- 🎯 **목표**: "실무에서 어떻게 적용하는가"

## 🎯 이 노트북에서 배우는 것들

### ✅ 실무급 워크플로우 패턴
1. **Routing**: EXAONE이 쿼리를 분석해서 적절한 경로로 라우팅
2. **Fan-out/Fan-in**: 병렬 처리로 효율성 극대화
3. **대화 기록 요약**: 긴 대화를 지능적으로 요약
4. **Human in the Loop**: AI 처리에 인간 검토 추가

### 🔄 패턴별 01번 연결
- **Routing** ← 01번 Command를 LLM 판단에 적용
- **Fan-out/Fan-in** ← 01번 Annotated State로 안전한 병렬 처리
- **Summarization** ← 새로운 패턴! 대화 메모리 관리
- **Human in the Loop** ← 01번 interrupt를 이메일 시스템에 적용

### 💼 실무 시나리오
- **고객 문의 자동 처리**: LLM이 문의 유형 판단 → 전문팀 자동 라우팅
- **다각도 정보 수집**: 3개 팀이 동시에 조사 → EXAONE이 통합 리포트 작성
- **챗봇 메모리 관리**: 긴 대화를 요약해서 컨텍스트 유지
- **이메일 자동 응답**: AI 초안 생성 → 인간 검토 → 최종 발송

### ⏱️ 예상 학습 시간: 30-40분
각 패턴을 EXAONE으로 직접 실행해보면서 실무 적용 방법을 체득합니다!

In [ ]:
# 필요한 라이브러리 설치 및 import + EXAONE 모델 로드
!pip install -q langgraph langchain langchain-teddynote
!pip install -q grandalf matplotlib networkx pyppeteer
!pip install -q git+https://github.com/lgai-exaone/transformers@add-exaone4
!pip install -q torch accelerate

from typing import TypedDict, List, Dict, Any, Annotated
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command
from langchain_core.messages import HumanMessage, AIMessage
from langchain_teddynote.graphs import visualize_graph
from langchain_teddynote.messages import stream_graph, random_uuid
import json
import random
import time
import torch
import gc
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 import 완료!")
print("🔥 실제 Human in the Loop을 위한 interrupt, Command 추가!")

# 🤖 EXAONE 모델 로드
MODEL_NAME = "LGAI-EXAONE/EXAONE-4.0-1.2B"

print(f"🚀 EXAONE-4.0-1.2B 모델 로드 시작: {MODEL_NAME}")

# EXAONE-4.0-1.2B 모델 및 토크나이저 로드 (CPU 호환)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,  # CPU 호환을 위해 float32 사용
    trust_remote_code=True
)

print("✅ EXAONE-4.0-1.2B 모델 로드 성공!")

# GPU 메모리 최적화
def clear_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def pure_exaone_inference(messages_or_prompt):
    """🔥 EXAONE 대화 맥락 지원 함수 - HuggingFace 가이드라인 준수"""
    clear_gpu_memory()
    
    try:
        # 🎯 입력이 문자열이면 단일 메시지로, 리스트면 대화 맥락으로 처리
        if isinstance(messages_or_prompt, str):
            messages = [{"role": "user", "content": messages_or_prompt}]
        else:
            # 이미 EXAONE chat format인 경우
            messages = messages_or_prompt
        
        # 🔥 EXAONE 표준 chat template로 전체 대화 맥락 처리
        input_ids = tokenizer.apply_chat_template(
            messages,  # 전체 대화 history
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=80,     # 더 짧고 집중된 응답
                temperature=0.1,       # 일관성을 위한 낮은 값
                do_sample=True,        # HuggingFace 가이드 권장
                top_p=0.9,            # 적절한 창의성 허용
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # 입력 길이만큼 제거하고 새로 생성된 부분만 추출
        generated_ids = outputs[0][input_ids.shape[-1]:]
        ai_response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        
        clear_gpu_memory()
        
        # 🔥 깔끔한 응답만 반환
        return ai_response if ai_response else "안녕하세요!"
        
    except Exception as e:
        clear_gpu_memory()
        print(f"🚨 EXAONE 추론 오류: {e}")
        return "죄송해요, 다시 말씀해 주시겠어요?"

print("✅ EXAONE 모델 설정 완료!")
print("🔥 최적화된 EXAONE 시스템:")
print("   💬 chat template: 전체 대화 history 처리")
print("   🎯 temperature=0.1 + do_sample=True: 일관성 + 창의성")
print("   🧠 max_new_tokens=80: 집중된 응답")
print("   🎪 top_p=0.9: 적절한 다양성")
print("💻 CPU 최적화 모드 (교육용 안정성 우선)")

## 🧠 Part 1: Routing 패턴 - LLM이 판단하는 지능적 라우팅

### 💡 01번에서 배운 것을 실무에 적용!
01번에서 **Command**로 다음 노드를 동적으로 선택하는 방법을 배웠어요.  
이제 **EXAONE LLM이 내용을 이해**해서 적절한 처리 경로를 결정하게 만들어봅시다!

### 🎯 Routing이란?
00번에서는 **글자 수**나 **단순 조건**으로 분기했지만,  
이제는 **LLM이 내용을 이해**해서 적절한 처리 경로를 결정해보자!

### 🔄 실무 시나리오
- **계산 문제**: 계산기로 라우팅
- **번역 요청**: 번역 엔진으로 라우팅  
- **일반 대화**: 대화 시스템으로 라우팅
- **코딩 질문**: 코드 분석기로 라우팅

**핵심**: LLM이 질문의 **의도**를 파악해서 최적의 처리 방법 선택!

### 🔗 01번 메커니즘 vs 02번 실무 적용
```
01번: Command(goto='next_node', update={...})
      ↓ "도구 사용법"
02번: EXAONE이 질문 분석 → intelligent_router가 Command 리턴
      ↓ "LLM 기반 실무 라우팅"
```

In [ ]:
# 🤖 Routing용 State 정의
class RoutingState(TypedDict):
    user_query: str          # 사용자 질문
    query_type: str          # LLM이 분석한 질문 유형
    processing_route: str    # 선택된 처리 경로
    result: str             # 최종 처리 결과
    confidence: float       # 라우팅 신뢰도

def intelligent_router(state: RoutingState) -> str:
    """🧠 EXAONE LLM이 질문을 분석해서 적절한 경로 결정"""
    user_query = state["user_query"]
    
    # 🤖 EXAONE에게 질문 분류 요청
    routing_prompt = f"""
다음 사용자 질문을 분석해서 가장 적절한 처리 방법을 선택해주세요:

질문: {user_query}

선택 가능한 경로:
1. "calculation" - 수학 계산이나 연산이 필요한 질문
2. "translation" - 번역이나 언어 변환이 필요한 질문
3. "coding" - 프로그래밍이나 코드 관련 질문
4. "general" - 일반적인 대화나 상식 질문

가장 적절한 경로를 하나만 선택해서 답변해주세요: """

    llm_response = pure_exaone_inference(routing_prompt)
    
    # 🔍 LLM 응답에서 라우팅 경로 추출
    if "calculation" in llm_response.lower():
        return "calculation"
    elif "translation" in llm_response.lower():
        return "translation" 
    elif "coding" in llm_response.lower():
        return "coding"
    else:
        return "general"

print("🧠 지능적 라우터 정의 완료!")
print("   - EXAONE LLM이 질문 내용을 분석")
print("   - 4가지 경로 중 최적 경로 선택")
print("   - 하드코딩 없는 완전 지능적 판단")

In [ ]:
# 🔧 각 경로별 처리 노드들 정의
def calculation_node(state: RoutingState) -> RoutingState:
    """🧮 계산 전용 노드 - EXAONE을 계산기로 활용"""
    user_query = state["user_query"]
    
    # EXAONE에게 계산 요청
    calc_prompt = f"""
다음 수학 계산을 정확히 계산해주세요:

계산: {user_query}

숫자와 기본 사칙연산만 사용해서 정확한 답을 제공해주세요.
예: 10 + 5 = 15
"""
    
    result = pure_exaone_inference(calc_prompt)
    
    return {
        **state,
        "query_type": "calculation",
        "processing_route": "계산 처리",
        "result": f"🧮 계산 결과: {result}",
        "confidence": 0.95
    }

def translation_node(state: RoutingState) -> RoutingState:
    """🌐 번역 전용 노드 - EXAONE을 번역기로 활용"""
    user_query = state["user_query"]
    
    translation_prompt = f"""
다음 텍스트를 번역해주세요:

원문: {user_query}

한국어면 영어로, 영어면 한국어로 번역해주세요.
정확하고 자연스러운 번역을 제공해주세요.
"""
    
    result = pure_exaone_inference(translation_prompt)
    
    return {
        **state,
        "query_type": "translation", 
        "processing_route": "번역 처리",
        "result": f"🌐 번역 결과: {result}",
        "confidence": 0.90
    }

def coding_node(state: RoutingState) -> RoutingState:
    """💻 코딩 전용 노드 - EXAONE을 프로그래밍 도우미로 활용"""
    user_query = state["user_query"]
    
    coding_prompt = f"""
다음 프로그래밍 질문에 답변해주세요:

질문: {user_query}

코드 예시와 함께 명확한 설명을 제공해주세요.
실용적이고 이해하기 쉬운 답변을 부탁드립니다.
"""
    
    result = pure_exaone_inference(coding_prompt)
    
    return {
        **state,
        "query_type": "coding",
        "processing_route": "코딩 처리", 
        "result": f"💻 코딩 답변: {result}",
        "confidence": 0.85
    }

def general_node(state: RoutingState) -> RoutingState:
    """💬 일반 대화 노드 - EXAONE을 친근한 대화 상대로 활용"""
    user_query = state["user_query"]
    
    general_prompt = f"""
다음 질문에 친근하고 도움이 되는 답변을 해주세요:

질문: {user_query}

자연스럽고 유용한 정보를 포함한 답변을 부탁드립니다.
"""
    
    result = pure_exaone_inference(general_prompt)
    
    return {
        **state,
        "query_type": "general",
        "processing_route": "일반 대화 처리",
        "result": f"💬 일반 답변: {result}",
        "confidence": 0.80
    }

print("🔧 모든 라우팅 처리 노드 정의 완료!")
print("   🧮 calculation_node: 수학 계산 전문")
print("   🌐 translation_node: 언어 번역 전문")  
print("   💻 coding_node: 프로그래밍 질문 전문")
print("   💬 general_node: 일반 대화 전문")
print("   🎯 각 노드가 특화된 EXAONE 프롬프트 사용")

In [ ]:
# 🔄 지능적 라우팅 워크플로우 구축
routing_workflow = StateGraph(RoutingState)

# 모든 처리 노드들 추가
routing_workflow.add_node("calculation", calculation_node)
routing_workflow.add_node("translation", translation_node) 
routing_workflow.add_node("coding", coding_node)
routing_workflow.add_node("general", general_node)

# 🧠 조건부 시작점: EXAONE이 분석해서 적절한 노드로 라우팅
routing_workflow.set_conditional_entry_point(
    intelligent_router,  # LLM 기반 라우팅 함수
    {
        "calculation": "calculation",
        "translation": "translation", 
        "coding": "coding",
        "general": "general"
    }
)

# 모든 처리 노드는 END로 연결 (단일 처리 후 종료)
routing_workflow.add_edge("calculation", END)
routing_workflow.add_edge("translation", END)
routing_workflow.add_edge("coding", END)
routing_workflow.add_edge("general", END)

# 워크플로우 컴파일
routing_app = routing_workflow.compile()

print("🔄 지능적 라우팅 워크플로우 완성!")
print("   🧠 EXAONE LLM이 질문 내용 분석")
print("   🎯 4개 전문 노드 중 최적 경로 자동 선택")  
print("   ⚡ 하드코딩 없는 완전 지능적 분기")
print("   💎 실무에서 바로 사용 가능한 패턴")

In [ ]:
# 📊 라우팅 워크플로우 시각화  
print("📊 지능적 라우팅 워크플로우 구조:")
visualize_graph(routing_app)

In [ ]:
# 🧪 지능적 라우팅 시스템 테스트
print("🧪 EXAONE 기반 지능적 라우팅 테스트")
print("=" * 60)

# 4가지 다른 유형의 질문으로 라우팅 테스트
routing_test_queries = [
    "10 + 25를 계산해줘",
    "안녕하세요를 영어로 번역해줘", 
    "파이썬에서 리스트를 어떻게 만드나요?",
    "오늘 날씨가 정말 좋네요!"
]

for i, query in enumerate(routing_test_queries, 1):
    print(f"\n🔍 테스트 {i}: \"{query}\"")
    print("-" * 40)
    
    # 라우팅 시스템 실행
    result = routing_app.invoke({
        "user_query": query,
        "query_type": "",
        "processing_route": "",
        "result": "",
        "confidence": 0.0
    })
    
    print(f"🧠 질문 유형 분석: {result['query_type']}")
    print(f"🛣️  선택된 처리 경로: {result['processing_route']}")
    print(f"📊 라우팅 신뢰도: {result['confidence']*100:.0f}%")
    print(f"✅ 처리 결과: {result['result']}")

print(f"\n🎯 지능적 라우팅 테스트 완료!")
print("💡 EXAONE LLM이 질문 내용을 이해하고 적절한 경로로 자동 분기!")
print("🔥 하드코딩 없이 완전히 지능적인 워크플로우 구현!")

# 라우팅 워크플로우의 핵심 장점 설명
print(f"\n{'🎯' * 20} 라우팅 패턴의 핵심 장점 {'🎯' * 20}")
print("✅ 완전 지능적 분기: LLM이 내용을 이해해서 판단")
print("✅ 확장성: 새로운 처리 경로 추가 시 코드 수정 최소화")
print("✅ 유지보수성: 각 처리 노드가 독립적으로 관리")
print("✅ 실무 적용성: 실제 서비스에서 바로 사용 가능한 구조")
print("✅ 성능 최적화: 각 작업에 특화된 프롬프트로 최적 성능")

## 🎯 실습 1: LLM 기반 라우터 직접 만들어보기

방금 본 `intelligent_router` 함수를 참고하여, 더 간단한 버전의 라우터를 작성해보세요.

**요구사항:**
- 사용자 쿼리에서 키워드를 찾아 적절한 노드로 라우팅
- "계산" 키워드 → "calc" 노드
- "번역" 키워드 → "translate" 노드
- 그 외 → "general" 노드
- LLM 호출 없이 간단한 조건문으로 구현

In [ ]:
from typing import TypedDict, Literal

class SimpleState(TypedDict):
    query: str
    result: str

def simple_keyword_router(state: SimpleState) -> Literal["calc", "translate", "general"]:
    '''
    사용자 쿼리를 분석하여 적절한 노드로 라우팅하는 함수
    '''
    query = state['query']
    
    # 여기를 수정하세요! 👇
    # 키워드 기반으로 라우팅 로직을 작성하세요
    pass  # 이 줄을 지우고 작성하세요

# 테스트
test_state = {"query": "10 곱하기 5를 계산해줘", "result": ""}
print(f"라우팅 결과: {simple_keyword_router(test_state)}")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
def simple_keyword_router(state: SimpleState) -> Literal["calc", "translate", "general"]:
    '''
    사용자 쿼리를 분석하여 적절한 노드로 라우팅하는 함수
    '''
    query = state['query']
    
    if '계산' in query or '더하기' in query or '곱하기' in query:
        return "calc"
    elif '번역' in query or 'translate' in query:
        return "translate"
    else:
        return "general"

# 테스트
test_state = {"query": "10 곱하기 5를 계산해줘", "result": ""}
print(f"라우팅 결과: {simple_keyword_router(test_state)}")  # calc
```

**핵심 개념:**
- Literal 타입으로 가능한 라우팅 대상을 명시
- 간단한 키워드 매칭으로 빠른 라우팅 가능
- LLM 호출 없이도 기본적인 의도 분류 가능

</details>

## 🔄 Part 2: Fan-out/Fan-in 패턴 - 병렬 처리로 효율 극대화

### 💡 01번에서 배운 것을 실무에 적용!
01번에서 **Annotated State**로 여러 노드의 결과를 안전하게 병합하는 방법을 배웠어요.  
이제 **실제 3개 전문팀이 동시에 EXAONE으로 조사**하고 결과를 통합해봅시다!

### 🎯 실무 시나리오
한 주제에 대해 다각도로 정보를 수집할 때:
- 🏛️ **역사팀**: 역사적 배경과 사건 조사
- 🎭 **문화팀**: 문화적 특징과 전통 조사
- 🗺️ **관광팀**: 관광지와 명소 조사

모든 팀이 동시에 작업하고 → EXAONE이 최종 통합 리포트 작성!

### 🔗 01번 메커니즘 vs 02번 실무 적용
```
01번: Annotated[List[str], operator.add]로 안전한 병합
      ↓ "도구 사용법"
02번: 3개 EXAONE 노드가 병렬 실행 → results 자동 누적
      ↓ "실무 병렬 처리"
```

In [ ]:

from typing import Annotated
import operator

class FanoutState(TypedDict):
    user_topic: str              # 사용자가 요청한 주제
    results: Annotated[List[str], operator.add]  # 병렬 결과들을 병합
    final_summary: str           # 최종 통합 결과

def distributor_node(state: FanoutState) -> FanoutState:
    """🚀 시작 노드 - 병렬 처리를 위한 분배"""
    return {
        **state,
        "results": []  # 결과 리스트 초기화
    }

# 🏛️ 역사 전문 노드
def history_research_node(state: FanoutState) -> FanoutState:
    """🏛️ 역사 정보를 전문적으로 조사"""
    topic = state["user_topic"]
    
    history_prompt = f"""
'{topic}'에 관련된 역사적 배경과 중요한 역사적 사건들에 대해 전문적으로 설명해주세요:

주제: {topic}

역사적 관점에서:
- 주요 역사적 사건
- 시대적 배경
- 역사적 의미와 영향

전문적이고 정확한 역사 정보를 제공해주세요.
"""
    
    result = pure_exaone_inference(history_prompt)
    
    return {
        "results": [f"🏛️ 역사 분야: {result}"]
    }

# 🎭 문화 전문 노드
def culture_research_node(state: FanoutState) -> FanoutState:
    """🎭 문화 정보를 전문적으로 조사"""
    topic = state["user_topic"]
    
    culture_prompt = f"""
'{topic}'에 관련된 문화적 특징과 전통에 대해 전문적으로 설명해주세요:

주제: {topic}

문화적 관점에서:
- 전통 문화와 풍습
- 예술과 문학
- 생활 문화와 특징

문화적 깊이가 있는 정보를 제공해주세요.
"""
    
    result = pure_exaone_inference(culture_prompt)
    
    return {
        "results": [f"🎭 문화 분야: {result}"]
    }

# 🗺️ 관광지 전문 노드  
def tourism_research_node(state: FanoutState) -> FanoutState:
    """🗺️ 관광 정보를 전문적으로 조사"""
    topic = state["user_topic"]
    
    tourism_prompt = f"""
'{topic}'에 관련된 관광지와 명소에 대해 전문적으로 설명해주세요:

주제: {topic}

관광 관점에서:
- 대표적인 명소와 관광지
- 특별한 체험과 활동
- 방문할만한 가치가 있는 곳들

실용적인 관광 정보를 제공해주세요.
"""
    
    result = pure_exaone_inference(tourism_prompt)
    
    return {
        "results": [f"🗺️ 관광 분야: {result}"]
    }

print("🔄 Fan-out 노드들 정의 완료!")
print("   🚀 distributor_node: 병렬 처리 시작점")
print("   🏛️ history_research_node: 역사 전문 조사")
print("   🎭 culture_research_node: 문화 전문 조사")
print("   🗺️ tourism_research_node: 관광 전문 조사")
print("   ⚡ Annotated 키로 안전한 병렬 결과 병합")

In [ ]:
# 🔗 Fan-in 노드 및 올바른 워크플로우 구축
def synthesis_node(state: FanoutState) -> FanoutState:
    """🔗 3개 전문 분야 결과를 종합해서 완전한 답변 생성"""
    topic = state["user_topic"]
    results = state.get("results", [])
    
    # 각 분야별 결과 정리
    all_results = "\n\n".join(results)
    
    # 전체 정보를 종합하는 EXAONE 프롬프트
    synthesis_prompt = f"""
다음 주제에 대한 3개 분야의 전문 조사 결과를 종합해서 완전하고 체계적인 답변을 만들어주세요:

주제: {topic}

조사 결과:
{all_results}

위 3개 전문 분야의 정보를 종합하여:
1. 전체적인 개요와 특징
2. 각 분야 간의 연관성과 통합적 관점
3. 종합적인 결론과 권장사항

체계적이고 완성도 높은 통합 답변을 제공해주세요.
"""
    
    final_result = pure_exaone_inference(synthesis_prompt)
    
    return {
        **state,
        "final_summary": f"🔗 종합 결과: {final_result}"
    }

# 🔄 Fan-out/Fan-in 워크플로우 구축 (LangGraph 표준 구조)
fanout_workflow = StateGraph(FanoutState)

# 1️⃣ 모든 노드 추가
fanout_workflow.add_node("distributor", distributor_node)    # 시작/분배 노드
fanout_workflow.add_node("history", history_research_node)   # 병렬 노드 1
fanout_workflow.add_node("culture", culture_research_node)   # 병렬 노드 2  
fanout_workflow.add_node("tourism", tourism_research_node)   # 병렬 노드 3
fanout_workflow.add_node("synthesis", synthesis_node)        # 통합 노드

# 2️⃣ 워크플로우 연결 (사용자 예시 패턴 적용)
fanout_workflow.add_edge(START, "distributor")    # START → distributor

# 3️⃣ Fan-out: distributor에서 3개 병렬 노드로 분기
fanout_workflow.add_edge("distributor", "history")
fanout_workflow.add_edge("distributor", "culture") 
fanout_workflow.add_edge("distributor", "tourism")

# 4️⃣ Fan-in: 3개 병렬 노드에서 synthesis로 통합
fanout_workflow.add_edge("history", "synthesis")
fanout_workflow.add_edge("culture", "synthesis")
fanout_workflow.add_edge("tourism", "synthesis")

# 5️⃣ 최종 종료
fanout_workflow.add_edge("synthesis", END)

# 컴파일
fanout_app = fanout_workflow.compile()

print("🔄 Fan-out/Fan-in 워크플로우 완성!")
print("   🚀 START → distributor: 단일 시작점")
print("   📤 Fan-out: distributor → {history, culture, tourism}")
print("   📥 Fan-in: {history, culture, tourism} → synthesis") 
print("   🏁 synthesis → END: 최종 완료")
print("   ⚡ LangGraph 표준 구조로 안전한 병렬 처리!")
print("   🎯 InvalidUpdateError 완전 해결!")

In [ ]:
# 📊 Fan-out/Fan-in 워크플로우 시각화
print("📊 병렬 처리 워크플로우 구조:")
visualize_graph(fanout_app)

In [ ]:
# 🧪 Fan-out/Fan-in 시스템 테스트 (수정된 구조)
print("🧪 병렬 처리 시스템 테스트")
print("=" * 50)

# 복합적인 주제로 테스트 (3개 분야 모두에서 정보가 나올 수 있는 주제)
test_topic = "한국"

print(f"🔍 테스트 주제: \"{test_topic}\"")
print("📤 Fan-out: 역사, 문화, 관광 3개 분야 병렬 조사 시작...")
print("-" * 50)

# Fan-out/Fan-in 시스템 실행 (새로운 State 구조)
result = fanout_app.invoke({
    "user_topic": test_topic,
    "results": [],
    "final_summary": ""
})

print("📥 병렬 처리 결과:")
print(f"\n🔗 수집된 전문 분야 결과 ({len(result['results'])}개):")
for i, res in enumerate(result['results'], 1):
    print(f"   {i}. {res}")

print(f"\n🔗 최종 통합 결과:")
print(f"   {result['final_summary']}")

print(f"\n🎯 Fan-out/Fan-in 테스트 완료!")
print("🔥 LangGraph 표준 구조로 완벽한 Fan-out/Fan-in 구현!")

# Fan-out/Fan-in 패턴의 핵심 장점 설명
print(f"\n{'🔄' * 20} Fan-out/Fan-in 패턴의 핵심 장점 {'🔄' * 20}")
print("⚡ 성능 향상: 병렬 처리로 처리 시간 단축")
print("🎯 품질 향상: 다각도 전문 분석으로 완성도 높은 답변")
print("🔧 안전성: Annotated 키로 상태 충돌 완전 방지")
print("📈 확장성: 새로운 전문 분야 추가 용이")
print("🎪 실무 적용: LangGraph 표준을 따르는 안정적 구조")

## 🎯 실습 2: Fan-out/Fan-in 이해하기

방금 본 병렬 처리 예시에서 3개의 노드가 동시에 실행되었습니다.

**질문:** Fan-out/Fan-in 패턴의 장점으로 옳지 않은 것은?

A) 여러 작업을 동시에 실행하여 전체 처리 시간을 단축할 수 있다  
B) 각 노드는 독립적으로 실행되므로 한 노드의 실패가 다른 노드에 영향을 주지 않는다  
C) LangGraph가 자동으로 상태를 병합하므로 개발자가 병합 로직을 작성할 필요가 없다  
D) 모든 노드가 완료될 때까지 기다리므로 가장 느린 노드의 처리 시간이 전체 시간을 결정한다

In [ ]:
# 여기에 답을 입력하세요
my_answer = ""  # "A", "B", "C", "D" 중 하나를 입력하세요

# 제출
print(f"선택한 답: {my_answer}")

<details>
<summary>📝 정답 보기 (클릭)</summary>

**정답: C**

**해설:**

A) ✅ 맞습니다. 병렬 실행으로 처리 시간을 단축할 수 있습니다.

B) ✅ 맞습니다. 각 노드는 독립적으로 실행됩니다.

C) ❌ **틀렸습니다.** LangGraph는 자동으로 Fan-in을 처리하지만, **개발자가 Annotated를 사용하여 병합 규칙을 명시해야 합니다.**

```python
# 개발자가 병합 규칙을 지정해야 함
class State(TypedDict):
    results: Annotated[List[str], operator.add]  # ← 리스트 누적
    query: str  # ← 마지막 업데이트로 덮어쓰기
```

D) ✅ 맞습니다. 모든 병렬 노드가 완료될 때까지 기다립니다.

**핵심:**
- Fan-in에서 Annotated 없는 필드는 마지막 업데이트로 덮어씀
- Annotated[List, operator.add]는 리스트를 누적
- 개발자가 명시적으로 병합 전략을 지정해야 함

</details>

## 📚 Part 3: 멀티턴 대화 & 요약 패턴 - 지능적 메모리 관리

### 🎯 멀티턴 대화에서 메모리 관리가 왜 중요한가?
여러 턴에 걸친 긴 대화가 계속되면 **토큰 한계**에 도달하고 **성능이 저하**됩니다.  
하지만 단순히 오래된 대화를 버리면 **중요한 맥락을 잃게** 됩니다.

### 💡 지능적 해결책: EXAONE 기반 요약
- **중요한 정보 보존**: 사용자 이름, 선호도, 핵심 대화 내용
- **불필요한 내용 제거**: 인사말, 반복적인 내용, 임시 정보  
- **맥락 유지**: 요약 후에도 자연스러운 대화 흐름 보장

### 🔄 실무 시나리오: 챗봇 멀티턴 대화
1. 고객과 여러 턴 대화 진행 (20턴 이상)
2. 대화가 길어지면 자동 요약 트리거
3. EXAONE이 핵심 내용만 추출해서 간단히 요약
4. 요약된 내용 + 최근 5턴으로 메모리 최적화
5. 성능 유지하면서 맥락도 보존하는 완벽한 균형!

### 🔗 새로운 패턴!
01번에서는 다루지 않은 실무 전용 패턴입니다.
- **멀티턴 대화 관리**: 여러 턴에 걸친 대화 추적
- **조건부 실행**: 대화 길이에 따라 요약 노드 활성화
- **메모리 최적화**: LLM 컨텍스트 창 관리
- **맥락 보존**: 요약 + 최근 대화로 균형

In [ ]:
# 📚 대화 요약용 State 정의
class SummarizationState(TypedDict):
    conversation_history: List[str]  # 전체 대화 기록
    summary: str                     # 요약된 핵심 내용  
    recent_turns: List[str]         # 최근 몇 턴의 대화
    user_input: str                 # 현재 사용자 입력
    response: str                   # AI 응답
    should_summarize: bool          # 요약 필요 여부

def should_summarize_check(state: SummarizationState) -> str:
    """🔍 요약이 필요한지 판단 (3턴 = 6개 항목 기준)"""
    history = state.get("conversation_history", [])
    current_count = len(history)
    
    if current_count >= 6:  # 3턴 이상이면 요약
        return "summarize"
    else:
        return "normal_chat"

def summarization_node(state: SummarizationState) -> SummarizationState:
    """📚 EXAONE을 사용해서 대화 기록을 지능적으로 요약"""
    history = state.get("conversation_history", [])
    
    # 기존 요약이 있다면 포함
    existing_summary = state.get("summary", "")
    summary_prefix = f"이전 요약: {existing_summary}\n\n" if existing_summary else ""
    
    # 전체 대화 내용 요약
    conversation_text = "\n".join(history)
    
    summarization_prompt = f"""
{summary_prefix}다음 대화 내용을 핵심만 간단히 요약해주세요:

대화 내용:
{conversation_text}

요약 시 포함해야 할 정보:
1. 사용자의 이름, 선호도 등 개인 정보
2. 중요한 질문과 답변의 핵심 내용
3. 진행 중인 주제나 작업
4. 향후 대화에 필요한 맥락 정보

불필요한 인사말이나 반복적인 내용은 제외하고, 핵심만 간단히 요약해주세요.
"""
    
    new_summary = pure_exaone_inference(summarization_prompt)
    
    # 최근 5개 항목만 유지 (나머지는 요약으로 대체)
    recent_turns = history[-5:] if len(history) >= 5 else history
    
    return {
        **state,
        "summary": new_summary,
        "recent_turns": recent_turns,
        "conversation_history": recent_turns,  # 메모리 최적화
        "should_summarize": False
    }

def normal_chat_node(state: SummarizationState) -> SummarizationState:
    """💬 일반적인 대화 처리 (요약 없이)"""
    user_input = state["user_input"]
    history = state.get("conversation_history", [])
    summary = state.get("summary", "")
    
    # 기존 요약 + 최근 대화 맥락으로 프롬프트 구성
    context = ""
    if summary:
        context += f"이전 대화 요약: {summary}\n\n"
        
    if history:
        recent_context = "\n".join(history[-5:])  # 최근 5개 항목
        context += f"최근 대화:\n{recent_context}\n\n"
        
    chat_prompt = f"{context}사용자: {user_input}\n\n위 맥락을 고려해서 자연스럽게 응답해주세요."
    
    response = pure_exaone_inference(chat_prompt)
    
    # 대화 기록에 현재 턴 추가
    updated_history = history + [f"사용자: {user_input}", f"AI: {response}"]
    
    return {
        **state,
        "conversation_history": updated_history,
        "response": response
    }

print("📚 멀티턴 대화 & 요약 시스템 노드 정의 완료!")
print("   🔍 should_summarize_check: 6개 항목 이상이면 요약")  
print("   📚 summarization_node: 전체 대화 → 요약 + 최근 5개만 유지")
print("   💬 normal_chat_node: 요약 + 최근 대화로 맥락 유지")
print("   🎯 간단명료한 요약 로직!")

In [ ]:
# 📚 대화 요약 워크플로우 구축
summarization_workflow = StateGraph(SummarizationState)

# 노드들 추가
summarization_workflow.add_node("normal_chat", normal_chat_node)
summarization_workflow.add_node("summarize", summarization_node)

# 조건부 시작: 요약 필요성 체크 후 적절한 노드로 라우팅
summarization_workflow.set_conditional_entry_point(
    should_summarize_check,
    {
        "normal_chat": "normal_chat",
        "summarize": "summarize"
    }
)

# 요약 후에는 일반 대화로 이어짐
summarization_workflow.add_edge("summarize", "normal_chat")
summarization_workflow.add_edge("normal_chat", END)

# 컴파일
summarization_app = summarization_workflow.compile()

print("📚 대화 요약 워크플로우 완성!")
print("   🔍 자동 길이 체크: 3턴 이상이면 자동 요약 (강의용)")
print("   📚 지능적 요약: EXAONE이 핵심 내용만 추출")
print("   💾 메모리 최적화: 요약 + 최근 5턴만 유지")
print("   🎯 성능과 맥락의 완벽한 균형!")

In [ ]:
# 🧪 4턴 대화 요약 시스템 테스트
print("🧪 EXAONE 기반 대화 요약 시스템 테스트")
print("=" * 60)

# 4턴 대화를 시뮬레이션하여 요약 기능 확실히 테스트
print("📝 4턴 대화 시뮬레이션을 통한 요약 테스트")
print("💡 3턴 후(6개 항목 후)에 요약이 트리거되도록 설정!")
print("-" * 60)

# 초기 상태 설정
test_state = {
    "conversation_history": [],
    "summary": "",
    "recent_turns": [],
    "user_input": "",
    "response": "",
    "should_summarize": False
}

# 4턴의 대화 시나리오 (요약이 확실히 발생하도록)
conversation_turns = [
    "안녕하세요! 저는 김민수라고 합니다.",
    "파이썬 프로그래밍을 배우고 싶은데 어떻게 시작하면 좋을까요?",
    "데이터 분석 분야에 관심이 있어서 판다스도 배우고 싶습니다.",
    "머신러닝도 함께 배울 수 있을까요?"
]

print("🔄 턴별 대화 진행 및 상태 변화 관찰:")
print("=" * 60)

for turn_num, user_input in enumerate(conversation_turns, 1):
    print(f"\n{'='*20} {turn_num}턴 시작 {'='*20}")
    print(f"📢 사용자 입력: \"{user_input}\"")
    
    before_history = test_state.get("conversation_history", [])
    
    # 현재 턴의 사용자 입력 설정
    test_state["user_input"] = user_input
    
    # 🔥 대화 시스템 실행 (디버그 출력 포함)
    result = summarization_app.invoke(test_state)
    
    # 결과 업데이트
    test_state = result
    
    after_history = result.get("conversation_history", [])
    print(f"🤖 AI 응답: \"{result.get('response', 'N/A')[:80]}...\"")
    
    # 요약 여부 상세 확인
    summary_content = result.get("summary", "")
    if summary_content:
        print(f"📚 ✅ 요약 생성 성공!")
        print(f"   📝 요약 내용: \"{summary_content[:100]}...\"")
        print(f"   💾 최근 대화 유지: {len(result.get('recent_turns', []))}개 항목")
        print(f"   🔥 메모리 최적화: {len(before_history)} → {len(after_history)}개로 압축!")
        print("   ✅ 요약 시스템 정상 작동!")
    else:
        print("   ⏳ 요약 없음 - 아직 조건 미달")
    
    # 다음 턴 예상
    expected_next = "요약 후 대화" if len(after_history) >= 6 else "일반 대화"
    print(f"   🔮 다음 턴 예상: {expected_next}")
    

print(f"\n{'🎯' * 25} 최종 테스트 결과 분석 {'🎯' * 25}")

# 최종 결과 검증
final_history = test_state.get("conversation_history", [])
final_summary = test_state.get("summary", "")

if final_summary:
    print("🎉 요약 시스템 테스트 성공!")
    print(f"   📊 최종 대화 기록: {len(final_history)}개")
    print(f"   📝 요약 길이: {len(final_summary)}글자")
    print(f"   💾 메모리 절약률: {((8-len(final_history))/8*100):.1f}% 절약")
    print(f"   🎯 맥락 보존: 요약으로 핵심 정보 유지")
else:
    print("❌ 요약 시스템 문제 발견!")
    print("   🔧 추가 디버깅이 필요합니다.")

print(f"\n{'🚀' * 25} 실무 적용 가이드 {'🚀' * 25}")
print("📈 확장성: 긴 고객 상담에서 토큰 한계 방지")
print("🧠 지능성: EXAONE이 핵심 내용만 선별하여 요약")
print("⚖️  균형성: 성능 최적화와 맥락 보존을 동시에 달성")
print("🔧 실용성: should_summarize_check에서 턴 수 쉽게 조정")
print("💡 강의 효과: 학생들이 요약 과정을 단계적으로 관찰 가능")

In [ ]:
# 📊 대화 요약 워크플로우 시각화
print("📊 대화 요약 워크플로우 구조:")
visualize_graph(summarization_app)

## 👥 Part 4: Human in the Loop 패턴 - 인간의 지혜가 필요한 순간

### 💡 01번에서 배운 것을 실무에 적용!
01번에서 **interrupt()**로 워크플로우를 일시정지하고 **Command(resume)**으로 재개하는 방법을 배웠어요.  
이제 **실제 이메일 자동 응답 시스템**에 적용해서 비즈니스 가치를 만들어봅시다!

### 🎯 Human in the Loop가 필요한 이유
AI가 아무리 똑똑해도 **중요한 고객 응대**나 **민감한 이메일**에는 인간의 검토가 필수!

### 📧 실제 비즈니스 시나리오: 이메일 자동 응답 시스템
현대 기업의 고객 서비스팀이 매일 수백 통의 고객 이메일을 처리하는 상황을 생각해보세요:

**🔄 Human in the Loop 워크플로우:**
1. **고객 이메일 접수**: 배송 지연, 환불 요청, 기술 문의 등
2. **AI 초안 생성**: EXAONE이 전문적이고 친절한 응답 초안 자동 작성
3. **인간 검토 단계**: 고객서비스팀이 AI 초안을 검토하고 피드백 제공
4. **최종 응답 확정**: 피드백 반영하여 완성된 응답을 고객에게 발송

### 💡 핵심 가치
- **효율성**: AI가 80% 작업을 자동화하여 처리 시간 대폭 단축
- **품질 보장**: 인간 전문가의 검토로 고객 만족도 향상
- **위험 관리**: 민감한 내용이나 복잡한 상황에서 실수 방지
- **일관성**: 회사 정책과 톤앤매너를 일관되게 유지

### 🚀 실무 적용 효과
- **처리량 300% 증가**: 하루 100통 → 300통 처리 가능
- **응답 품질 향상**: 인간 검토로 고객 만족도 95% 달성
- **비용 절감**: 인력 투입은 줄이고 서비스 품질은 향상

### 🔗 01번 메커니즘 vs 02번 실무 적용
```
01번: interrupt() → Command(resume) 기본 동작
      ↓ "도구 사용법"
02번: 이메일 초안 → 검토자 피드백 → 최종 발송
      ↓ "실무 이메일 시스템"
```

아래에서 실제 작동하는 Human in the Loop 시스템을 체험해보세요!

In [ ]:
from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph, add_messages
from langgraph.types import interrupt, Command
from langchain.schema import HumanMessage

# 📧 이메일 자동 응답 시스템용 State
class EmailState(TypedDict):
    customer_email: str      # 고객 이메일 내용
    ai_draft: str           # AI가 생성한 응답 초안
    human_feedback: str     # 인간 검토자 피드백
    final_response: str     # 최종 이메일 응답

def ai_draft_node(state: EmailState):
    """🤖 AI가 고객 이메일에 대한 응답 초안 생성"""
    customer_email = state["customer_email"]
    
    prompt = f"""
다음 고객 이메일에 대한 전문적이고 친절한 응답을 작성해주세요:

고객 이메일: {customer_email}

회사 정책에 맞는 정중하고 도움이 되는 응답을 작성해주세요.
"""
    
    ai_draft = pure_exaone_inference(prompt)
    print(f"🤖 AI 응답 초안 생성 완료")
    return {"ai_draft": ai_draft}

def human_review_node(state: EmailState):
    """👤 인간 검토자가 AI 초안을 검토"""
    ai_draft = state["ai_draft"]
    
    print(f"📧 AI 응답 초안: {ai_draft}")
    
    # 🛑 여기서 인간 검토자의 승인/피드백 대기
    feedback = interrupt({
        "message": "AI가 이메일 응답 초안을 생성했습니다. 검토 후 피드백을 주세요:",
        "draft": ai_draft
    })
    
    return {"human_feedback": feedback}

def finalize_node(state: EmailState):
    """✅ 최종 이메일 응답 확정"""
    ai_draft = state["ai_draft"]
    feedback = state["human_feedback"]
    
    if "승인" in feedback:
        final_response = ai_draft
    else:
        # 피드백 반영해서 수정
        prompt = f"""
다음 AI 초안을 검토자 피드백에 따라 수정해주세요:

원본 초안: {ai_draft}
검토자 피드백: {feedback}

피드백을 반영한 개선된 응답을 작성해주세요.
"""
        final_response = pure_exaone_inference(prompt)
    
    print(f"✅ 최종 이메일 응답 확정!")
    return {"final_response": final_response}

print("📧 이메일 자동 응답 Human in the Loop 시스템 구성 완료!")

In [ ]:
# 📧 이메일 자동 응답 워크플로우 생성
workflow = StateGraph(EmailState)

# 노드 추가
workflow.add_node("ai_draft", ai_draft_node)
workflow.add_node("human_review", human_review_node)
workflow.add_node("finalize", finalize_node)

# 워크플로우 연결
workflow.add_edge(START, "ai_draft")
workflow.add_edge("ai_draft", "human_review")
workflow.add_edge("human_review", "finalize")
workflow.add_edge("finalize", END)

# 체크포인터 설정 (Human in the Loop에 필수)
checkpointer = MemorySaver()
email_app = workflow.compile(checkpointer=checkpointer)

print("✅ 이메일 자동 응답 시스템 준비 완료!")

In [ ]:
# 📧 1단계: 고객 이메일 처리 (interrupt 지점까지)
import uuid

# 고객 이메일 예시
customer_email = """
안녕하세요,

지난주에 주문한 노트북이 아직 도착하지 않았습니다. 
주문번호는 ORD-2024-1234이고, 배송 예정일이 3일 전이었는데
아직 배송 추적에서 확인이 안 됩니다.

언제쯤 받을 수 있을지 알려주시면 감사하겠습니다.

김고객 드림
"""

print("📧 고객 이메일:")
print(customer_email)
print("\n" + "="*50)

# 워크플로우 실행
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

result = email_app.invoke(
    {"customer_email": customer_email}, 
    config=config
)

# interrupt 확인
if "__interrupt__" in result:
    print("✅ Human in the Loop 활성화!")
    print("👤 검토자의 피드백이 필요합니다.")
    print("\n📋 AI 초안:")
    print(result["ai_draft"])
    print(f"\n🔍 Thread ID: {thread_id}")
    print("📌 다음 셀에서 피드백을 제공하세요!")
else:
    print("❌ interrupt가 발생하지 않았습니다.")
    print("결과:", result)

In [ ]:
# 📧 2단계: 검토자 피드백 후 최종 응답 생성

# 검토자 피드백 (실제로는 웹 인터페이스에서 받음)
reviewer_feedback = """
AI 초안이 전반적으로 좋습니다. 
다만 다음 내용을 추가해주세요:
1. 사과 인사를 더 명확히
2. 구체적인 해결 방안 제시
3. 고객 만족을 위한 추가 혜택 언급

승인은 수정 후에 하겠습니다.
"""

print("👤 검토자 피드백:")
print(reviewer_feedback)
print("\n" + "="*50)

# Command(resume)으로 워크플로우 재시작
final_result = email_app.invoke(
    Command(resume=reviewer_feedback),
    config  # 이전과 동일한 config 사용
)

print("✅ 최종 이메일 응답 완성!")
print("\n📧 최종 고객 응답:")
print("="*60)
print(final_result["final_response"])
print("="*60)

print(f"\n🎉 Human in the Loop 프로세스 완료!")
print("💼 실무에서는 이 응답이 고객에게 자동 발송됩니다.")

In [ ]:
# 📧 Human in the Loop 워크플로우 정리

print("📋 이메일 자동 응답 Human in the Loop 시스템:")
print("=" * 60)

print("""
🎯 실제 비즈니스 시나리오:
   고객 → 배송 지연 문의 이메일 발송
   
📍 1단계 (AI 초안 생성):
   🤖 EXAONE이 고객 이메일 분석
   📝 전문적인 응답 초안 자동 생성
   
📍 2단계 (Human 검토):
   🛑 interrupt() 발생 → 워크플로우 중단
   👤 고객서비스팀 검토자가 초안 확인
   💬 피드백: "사과 인사 추가, 구체적 해결책 제시"
   
📍 3단계 (최종 응답):
   🔄 AI가 피드백 반영하여 응답 수정
   ✅ 최종 승인된 이메일 고객에게 발송

💡 핵심 가치:
   - 자동화로 효율성 증대
   - 인간 검토로 품질 보장
   - 고객 만족도 향상
""")

print("\n🔧 워크플로우 구조:")
visualize_graph(email_app)

print("\n🎓 학습 포인트:")
print("- interrupt()로 워크플로우 중단")
print("- Command(resume)로 재시작")
print("- 체크포인터로 상태 저장")
print("- 실무 적용 가능한 패턴")

## 🎯 실습 3: Human-in-the-Loop 구현하기

방금 본 이메일 검토 예시를 참고하여, 승인 워크플로우를 만들어보세요.

**시나리오:**
- 사용자가 문서를 제출하면 AI가 자동 검토
- 위험도가 높으면(risk_level > 7) 사람의 승인 필요
- interrupt()로 워크플로우를 일시 중지하고 사람의 피드백 대기

**요구사항:**
- ApprovalState에 risk_level, document, human_decision 필드 사용
- check_risk_node: 위험도가 7 초과면 needs_approval=True
- approval_node: needs_approval이면 interrupt로 중단하고 피드백 대기

In [ ]:
from typing import TypedDict
from langgraph.types import interrupt

class ApprovalState(TypedDict):
    document: str
    risk_level: int
    needs_approval: bool
    human_decision: str

def check_risk_node(state: ApprovalState):
    '''문서의 위험도를 확인'''
    risk = state['risk_level']
    needs_approval = risk > 7
    return {"needs_approval": needs_approval}

def approval_node(state: ApprovalState):
    '''사람의 승인이 필요하면 interrupt로 중단'''
    # 여기를 수정하세요! 👇
    # needs_approval이 True면 interrupt() 호출하여 피드백 대기
    # interrupt 메시지에 문서 정보와 위험도 포함
    pass  # 이 줄을 지우고 작성하세요

# ⚠️ 참고: interrupt()는 그래프 안에서만 작동합니다!
# 직접 호출하면 에러가 나요. 정답 보기에서 전체 그래프 예시를 확인하세요.
print("✅ approval_node 함수 정의 완료")
print("💡 interrupt()는 그래프로 컴파일한 후에만 작동합니다")

<details>
<summary>📝 정답 보기 (클릭)</summary>

```python
def approval_node(state: ApprovalState):
    '''사람의 승인이 필요하면 interrupt로 중단'''
    if state['needs_approval']:
        # 워크플로우를 중단하고 사람의 피드백 대기
        feedback = interrupt({
            "message": "위험도가 높은 문서입니다. 승인이 필요합니다.",
            "document": state["document"],
            "risk_level": state["risk_level"]
        })
        return {"human_decision": feedback}
    else:
        return {"human_decision": "자동 승인"}
```

**⚠️ 중요: interrupt()는 그래프 안에서만 작동!**

위 함수를 직접 호출하면 `RuntimeError: Called get_config outside of a runnable context` 에러가 발생합니다.

**올바른 사용법 (전체 그래프):**

```python
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# 1. 그래프 구성
workflow = StateGraph(ApprovalState)
workflow.add_node("check_risk", check_risk_node)
workflow.add_node("approval", approval_node)

workflow.set_entry_point("check_risk")
workflow.add_edge("check_risk", "approval")
workflow.add_edge("approval", END)

# 2. MemorySaver로 컴파일 (interrupt 사용 시 필수!)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# 3. 실행
config = {"configurable": {"thread_id": "approval-1"}}
result = app.invoke({
    "document": "기밀 문서 A",
    "risk_level": 9,
    "needs_approval": False,  # 이건 check_risk_node가 설정함
    "human_decision": ""
}, config=config)

print(result)
# interrupt()가 호출되면 워크플로우가 중단됨!

# 4. 재개 (사람이 검토 후)
from langgraph.types import Command
final = app.invoke(Command(resume="승인함"), config=config)
print(final)
```

**핵심 개념:**

1. **interrupt()는 그래프 컨텍스트 필수**: 
   - 노드를 직접 호출 ❌ → `approval_node(state)` 에러!
   - 그래프로 실행 ✅ → `app.invoke(...)` 정상 작동

2. **MemorySaver 필수**:
   - interrupt()로 중단된 상태를 저장하려면 checkpointer 필요
   - `compile(checkpointer=MemorySaver())`

3. **재개 방법**:
   - `app.invoke(Command(resume="사람의 응답"), config=config)`
   - 같은 thread_id 사용해야 이어서 실행됨

4. **사용 사례**:
   - 중요한 결정에 사람의 개입 필요
   - 외부 시스템 대기 (결제 승인, API 응답 등)
   - 단계별 검토 워크플로우

</details>

## 🎓 LangGraph 기본 패턴 학습 완료!

### ✅ 성공적으로 마스터한 4가지 핵심 패턴:

#### 1️⃣ **Routing 패턴** 🧠
- **목적**: 사용자 입력을 지능적으로 분류하여 적절한 처리 경로로 라우팅
- **실무 활용**: 고객 문의 유형별 자동 분류, 우선순위 설정
- **핵심 기술**: 조건부 엣지, LLM 기반 분류

#### 2️⃣ **Fan-out/Fan-in 패턴** 🔄  
- **목적**: 복잡한 작업을 여러 전문 영역으로 병렬 분산 후 결과 통합
- **실무 활용**: 다각도 분석, 전문팀 협업, 성능 최적화
- **핵심 기술**: 병렬 노드 실행, 결과 집계

#### 3️⃣ **Summarization 패턴** 📚
- **목적**: 긴 대화나 문서를 핵심 정보만 압축하여 메모리 효율성 향상
- **실무 활용**: 채팅봇 컨텍스트 관리, 회의록 요약, 문서 압축
- **핵심 기술**: 조건부 요약, 상태 관리

#### 4️⃣ **Human in the Loop 패턴** 👥
- **목적**: 중요한 결정이나 민감한 작업에 인간의 검토와 승인 과정 추가
- **실무 활용**: 이메일 자동 응답, 고객 서비스, 콘텐츠 검토
- **핵심 기술**: interrupt() 함수, 워크플로우 중단/재시작

### 🚀 다음 단계: 고급 통합 패턴
기본 패턴을 완전히 이해했다면, `01-advanced-integration.ipynb`에서 모든 패턴을 조합한 실무급 시스템을 구축해보세요!

### 💡 실무 적용 가이드
각 패턴은 독립적으로도 강력하지만, 조합할 때 진정한 비즈니스 가치를 발휘합니다:
- **단일 패턴**: 특정 문제 해결
- **패턴 조합**: 완전한 엔터프라이즈 솔루션

🎉 **축하합니다! 이제 LangGraph로 실무급 AI 워크플로우를 구축할 수 있습니다!**